Download dl data

In [8]:
import gc
import os
import time
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [9]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [10]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 

In [ ]:
pwd

In [12]:
country_code = 'ID'
start_date = 20190101
end_date = 20190102

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Start timing
start_time = time.time()

current_date = start_date_dt
while current_date <= end_date_dt:
    formatted_date = current_date.strftime('%Y%m%d')
    
    query = f"""
        SELECT 
            -- *,
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            timezoneoffset_secs,
            lat,
            lng, 
            speed_ms, 
            
            -- Extract the date and time part and adjust by the timezone offset 
            date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
            interval '1' second * timezoneoffset_secs AS event_datetime_utc
            
        FROM {pe_dl_table}
        WHERE 
            processing_date = {formatted_date} 
            AND country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
    """
    
    pe_dl_table_df = sql_engine.read_sql(query)
    
    file_path = f'/home/jovyan/Data/DL/{country_code}/{formatted_date}_{country_code}_pe_dl.csv'

    try:
        # Attempt to save the DataFrame to the CSV file
        pe_dl_table_df.to_csv(file_path, index=False)
    except OSError as e:
        # If there is an OSError (e.g., directory does not exist), create the directory
        if not os.path.exists(os.path.dirname(file_path)):
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            # Retry saving the DataFrame to the CSV file
            pe_dl_table_df.to_csv(file_path, index=False)
        else:
            # If the error is not related to the directory not existing, re-raise the exception
            raise e
        
    # pe_dl_table_df.to_csv(f'/home/jovyan/Data/DL/{country_code}/{formatted_date}_{country_code}_pe_dl.csv', index=False)
    
    # Delete the DataFrame to free up memory
    del pe_dl_table_df
    # Force garbage collection
    gc.collect()
    
    current_date += timedelta(days=1)

# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


Total time taken: 466.7766478061676 seconds


In [ ]:
# from datetime import datetime, timedelta

# country_code = 'MX'
# start_date = 20190101
# end_date = 20190131

# # Convert integer dates to datetime objects
# start_date = datetime.strptime(str(start_date), '%Y%m%d')
# end_date = datetime.strptime(str(end_date), '%Y%m%d')

# # Iterate through each day
# current_date = start_date
# while current_date <= end_date:
#     print(current_date.strftime('%Y%m%d'))
#     current_date += timedelta(days=1)